![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Callysto’s Weekly Data Visualization

## Government Spending

### Recommended Grade levels:
<br>

In [ ]:
import pandas as pd
import plotly.express as px
import folium
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

print("Libraries imported.")

In [ ]:
# Renaming columns
expenses_cols = ['Year','Social protection','Health','Education','General public services','Economic affairs','Other functions']
share_of_expenses_cols = ['Province','Health','Education','General public services','Social protection','Economic affairs','Other functions']

fte = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/government-spending/FTE.csv")
expenditures = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/government-spending/expenditures.csv")
overall_expenses = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/government-spending/overall_expenses.csv", header=1, names=expenses_cols)
planned_spending = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/government-spending/planned_spending.csv")
share_of_expenses = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/government-spending/share_of_expenses.csv", header=1, names=share_of_expenses_cols)

print("Datasets imported.")

In [ ]:
fte

In [ ]:
# Find unique organizations
unique_organizations = set(fte['Organization'])

# Print unique organization
for org in unique_organizations:
    print(org)

In [ ]:
# Change this to the organization you'd like to look at 
# Example: "Atlantic Canada Opportunities Agency" can be changed to "Canadian Grain Commission"
organization_to_find = "Atlantic Canada Opportunities Agency"

searched_df = fte[fte['Organization'] == organization_to_find]

columns_to_melt = [col for col in searched_df.columns if col != 'Program']
melted_df = pd.melt(searched_df, id_vars=['Organization'], value_vars=columns_to_melt, var_name='Year', value_name='Value')
org_df = melted_df.groupby(['Year', 'Organization'])['Value'].sum().reset_index()

searched_df = fte[fte['Organization'] == organization_to_find]
df_prog = searched_df.groupby('Program').sum().reset_index()
columns_to_melt_prog = [col for col in df_prog.columns if col != 'Program']
program_df = pd.melt(df_prog, id_vars=['Program'], value_vars=columns_to_melt_prog, var_name='Year', value_name='Value')

fte_fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Total FTEs for {organization_to_find}", f"Program FTEs for {organization_to_find}"))

for org in org_df['Organization'].unique():
    org_data = org_df[org_df['Organization'] == org]
    fte_fig.add_trace(go.Scatter(x=org_data['Year'], y=org_data['Value'], mode='lines',
                             name=f'{org}'), 
                             row=1, col=1)

for program in program_df['Program'].unique():
    prog_data = program_df[program_df['Program'] == program]
    fte_fig.add_trace(go.Scatter(x=prog_data['Year'], y=prog_data['Value'], mode='lines',
                             name=f'{program}'), row=1, col=2,
                             )
   
    

fte_fig.update_layout(title=f'Progression of FTEs for: {organization_to_find}',
                  xaxis_title='Year', yaxis_title='FTEs',
                  xaxis2_title='Year', yaxis2_title='FTEs')

fte_fig.show()

In [ ]:
expenditures

In [ ]:
print(expenditures.dtypes[expenditures.dtypes == 'float64'])
print(expenditures.dtypes[expenditures.dtypes == 'int64'])

In [ ]:
expenditures = expenditures.set_index('Organization')

# List of years
years = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22']

for year in years:
    year_col = year + ' - '
    budget_col = year_col + 'Total budgetary authority available for use'
    expenditure_col = year_col + 'Expenditures'
    
    expenditures[f'{year} Delta'] = expenditures[budget_col] - expenditures[expenditure_col]

expenditures = expenditures.reset_index()
display(expenditures)

In [ ]:
cols_to_check = expenditures.columns[12:]
max_indices = expenditures[cols_to_check].idxmax()

for col, max_index in max_indices.iteritems():
    max_value = expenditures.at[max_index, col]
    organization = expenditures.at[max_index, 'Organization']
    program = expenditures.at[max_index, 'Vote 2021-22 / Statutory - Description']  
    
    print(f"Highest {col}:")
    print(f"Organization: {organization}")
    print(f"Description: {program}")
    print(f"Under-Budget by: ${max_value}\n")

cols_to_check = expenditures.columns[12:]
min_indices = expenditures[cols_to_check].idxmin()
print('-----------------------------\n')

for col, min_index in min_indices.iteritems():
    min_value = expenditures.at[min_index, col]
    organization = expenditures.at[min_index, 'Organization']
    program = expenditures.at[min_index, 'Vote 2021-22 / Statutory - Description']  
    
    print(f"Highest {col}:")
    print(f"Organization: {organization}")
    print(f"Description: {program}")
    print(f"Over-Budget by: ${abs(min_value)}\n") 